In [12]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [13]:
teams = pd.read_csv('teams_table.csv')

In [3]:
sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': api_key,
    'sport': 'basketball_ncaab',
    'region': 'us',
    'oddsFormat': 'american'
})


# fbs_response = requests.get('https://api.fastbreakstats.com/teams')
# fbs_data = json.loads(fbs_response.text)

sports_json = json.loads(sports_response.text)
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json', 'w') as outfile:
    json.dump(sports_json, outfile)
# with open('odds.json') as f:
#     sports_json = json.load(f)

In [4]:
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json') as f:
    sports_json = json.load(f)

In [5]:
def getTeamData(name):
    name = name.replace(" ", "%20")
    baseUrl="https://api.fastbreakstats.com/teams/"
    params="?q=current"
    url=baseUrl+name+params

    r = requests.get(url = url).json()[0]
    bpival = r['BPIval']
    kpval = r['KenPomVal']
    srsval = r['SRS']
    return kpval, bpival, srsval

In [6]:
loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
loaded_model.coef_

array([[-0.14904683,  0.02408791, -0.00285878,  0.14617775, -0.05714497,
         0.03016929]])

In [7]:
def get_needed_odds1(conf):
    payout = round(((.1+(1-conf))/conf),4)
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [8]:
def get_ev1(odds,conf):
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
        
    ev = (conf * payout) - (1-conf)
    return ev

In [19]:
for game in sports_json['data']:
    #Loop through and get odds
    odds = ['n/a','n/a']
    for site in game['sites']:
        if site['site_key'] == 'bovada':
            odds = site['odds']['h2h']
            sitex = "bovada"
            break
        if site['site_key'] == 'draftkings':
            odds = site['odds']['h2h']
            sitex = "DK"
        if site['site_key'] == 'williamhill_us':
            odds = site['odds']['h2h']
            sitex = "WH"
            
    if odds == ['n/a','n/a']:
#         print("There are no odds for {0}".format(game['teams']))
        continue
    
    home = game['home_team']
    if game['home_team'] == game['teams'][0]:
        away = game['teams'][1]
        home_odds = odds[0]
        away_odds = odds[1]
    else:
        away = game['teams'][0]
        home_odds = odds[1]
        away_odds = odds[0]
    
    try:
        home = home.replace("-", " ")
        home = home.replace("(", "")
        home = home.replace(")", "")
        home = home.replace(".", "")
        away = away.replace("-", " ")
        away = away.replace("(", "")
        away = away.replace(")", "")
        away = away.replace(".", "")
        homekp, homebpi, homesrs = getTeamData(teams.loc[teams['odds_api_name'] == home]['kenpom_name'].values[0])
        awaykp, awaybpi, awaysrs = getTeamData(teams.loc[teams['odds_api_name'] == away]['kenpom_name'].values[0])
    except Exception as e:
        print(e)
        print(game['teams'])
        pass
        
    res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
#     print(res[0])
#     print("{0} & {1} from {2}".format(away_odds,home_odds,sitex))
    away_needed_odds = get_needed_odds1(res[0][0])    
    home_needed_odds = get_needed_odds1(res[0][1])
    away_ev = get_ev1(away_odds,res[0][0])
    home_ev = get_ev1(home_odds,res[0][1])
    if away_odds > away_needed_odds:
        print("{0} @ {1}".format(away,home))
        print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(away,away_needed_odds,away_odds,away_ev))
        print()
    if home_odds > home_needed_odds:
        print("{0} @ {1}".format(away,home))
        print("{0} needs {1} and has {2} odds with an Expected Value of {3}".format(home,home_needed_odds,home_odds,home_ev))
        print()

Wagner Seahawks @ Bryant Bulldogs
Wagner Seahawks needs 513.0 and has 525 odds with an Expected Value of 0.12065987536055789

UTSA Roadrunners @ Oregon St Beavers
UTSA Roadrunners needs 262.0 and has 295 odds with an Expected Value of 0.2014069086278506

San Diego Toreros @ Cal Poly Mustangs
San Diego Toreros needs -312.0 and has -245 odds with an Expected Value of 0.1734189673909758

Pittsburgh Panthers @ Miami Hurricanes
Miami Hurricanes needs -351.0 and has 112 odds with an Expected Value of 0.8153901307761028

Mercer Bears @ Georgia St Panthers
Georgia St Panthers needs -215.0 and has -200 odds with an Expected Value of 0.12558886029829902

TCU Horned Frogs @ Oklahoma St Cowboys
Oklahoma St Cowboys needs -251.0 and has -220 odds with an Expected Value of 0.14412632631371186

Duke Blue Devils @ Notre Dame Fighting Irish
Duke Blue Devils needs -222.0 and has -180 odds with an Expected Value of 0.17893064307556084

